In [1]:
'''手动读取.h5文件赋值给torch'''
import keras
from models.DFCAN16 import DFCAN as keras_DFCAN
from model.DFCAN import DFCAN as torch_DFCAN
input_height, input_width, n_channel = 502, 502, 1

weight_file = "model_weights.h5"

keras_model = keras_DFCAN((input_height, input_width, n_channel))
keras_model.load_weights(weight_file)

Using TensorFlow backend.
/root/miniconda3/envs/k2t_new/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#加载keras中可以学习的层的权重
keras_layers_name = []
keras_layers_weight = []
for layer in keras_model.layers:
    if layer.trainable == True:
        keras_layers_name.append(layer.name)
        keras_layers_weight.append(layer.weights)

In [3]:
print(len(keras_layers_weight))
print(len(keras_model.weights))


203
166


In [4]:
keras_model.weights[0]

<tf.Variable 'conv2d_1/kernel:0' shape=(3, 3, 1, 64) dtype=float32_ref>

In [5]:
def get_param(model, find_name):
    for name, param in model.named_parameters():
        if name == find_name:
            return param

In [6]:
#加载pytorch权重
torch_layers_names = []
torch_layers_params = []
model =  torch_DFCAN((n_channel))
for name, param in model.named_parameters():
    torch_layers_names.append(name)
    if param.requires_grad:
        # print(name, param.data)
        torch_layers_params.append(param)
# assert len(keras_layers_weight) == len(torch_layers_params)
pass

In [7]:
torch_layers_names[0]

'input.0.weight'

In [8]:
print(torch_layers_params[0].shape)
print(keras_model.get_weights()[0].shape)


torch.Size([64, 1, 3, 3])
(3, 3, 1, 64)


In [9]:
import torch

In [10]:
print(torch_layers_params[1].shape)
print(keras_model.get_weights()[1].shape)

torch.Size([64])
(64,)


In [11]:
#转第一层的权重,包含卷积核和偏置的
weight = keras_model.get_weights()[0]   #keras第一层权重：numpy.ndarray
to_torch_weight = torch.Tensor(weight).permute(3,2,0,1)    
# to_torch_weight = torch.Tensor(weight).permute(3,2,1,0)   
[k for k in model.named_parameters()][0][1].data = to_torch_weight
print(to_torch_weight[0,0,:,:])

tensor([[-0.0293,  0.0582, -0.0314],
        [ 0.0262, -0.0675, -0.0199],
        [ 0.0615, -0.0492,  0.0054]])


In [12]:
#转置偏置的参数
weight = keras_model.get_weights()[1]
to_torch_weight = torch.Tensor(weight)
[k for k in model.named_parameters()][1][1].data = to_torch_weight
weight.shape
to_torch_weight.shape

torch.Size([64])

In [13]:
#查看卷积核
# print(keras_model.get_weights()[0])
print(keras_model.get_weights()[0][:,:,0,0].shape)
print([k for k in model.named_parameters()][0][1][0,0,:,:].data.shape)
#查看权重
print(keras_model.get_weights()[0][:,:,0,0])    #keras卷积核
print([k for k in model.named_parameters()][0][1][0,0,:,:].data)    #torch卷积核
#权重除了精度不一致没什么问题

(3, 3)
torch.Size([3, 3])
[[-0.02934367  0.05818463 -0.0313947 ]
 [ 0.02616983 -0.06751775 -0.01991458]
 [ 0.06146571 -0.04916605  0.00540901]]
tensor([[-0.0293,  0.0582, -0.0314],
        [ 0.0262, -0.0675, -0.0199],
        [ 0.0615, -0.0492,  0.0054]])


In [14]:
#查看偏置
print(keras_model.get_weights()[1])    #keras卷积核
print([k for k in model.named_parameters()][1][1].data)    #torch卷积核

[ 0.00372856 -0.0246051   0.00285443  0.00414012 -0.0039361   0.00487928
  0.00389124 -0.00728025  0.00646339 -0.01064683 -0.00736134 -0.00355408
 -0.01244564  0.00290518 -0.00321625  0.00257412 -0.00024484  0.00878443
 -0.0122802   0.00623679 -0.00490301 -0.00498458  0.00059079 -0.00078095
  0.01566281  0.00461143  0.00481706  0.00191404  0.00436323  0.00103071
  0.00120827  0.0079076  -0.00237806 -0.00501676  0.0162858  -0.00070751
 -0.00639688 -0.00274556 -0.00539504 -0.00232929 -0.01197647 -0.01163186
 -0.0027836   0.00847736 -0.01085964  0.00859594 -0.00164084  0.00085174
  0.00040566  0.01297158  0.01043573 -0.00089228 -0.00121842  0.00931372
 -0.00452846 -0.00567043 -0.00110579  0.00754591  0.00065435 -0.01508514
 -0.00659712 -0.00016761 -0.00811328 -0.01486146]
tensor([ 0.0037, -0.0246,  0.0029,  0.0041, -0.0039,  0.0049,  0.0039, -0.0073,
         0.0065, -0.0106, -0.0074, -0.0036, -0.0124,  0.0029, -0.0032,  0.0026,
        -0.0002,  0.0088, -0.0123,  0.0062, -0.0049, -0.0050

In [15]:
#生成固定输入
torch.manual_seed(123)
x0 = torch.randn(1, 1, 502, 502)

In [16]:
#检查卷积层输出是否一致
#pytorch
torch_out = model.input[0](x0)

#keras
from keras import backend as K
# keras_x0 = K.constant(x0.permute(0,2,3,1).numpy())  #转到keras的输入
keras_x0 = x0.permute(0,2,3,1).numpy()
print(type(keras_x0))
print(keras_x0.shape)
from keras.models import Model
layer_name = 'conv2d_1'
intermediate_layer_model = Model(inputs=keras_model.input, outputs=keras_model.get_layer(layer_name).output)
keras_output = intermediate_layer_model.predict(keras_x0)
print(type(keras_output))
keras_output_torch_tensor = torch.Tensor(keras_output).permute(0, 3, 1, 2)

#比较二者
assert torch_out.shape == keras_output_torch_tensor.shape
print(torch_out == keras_output_torch_tensor)
print(torch_out.shape, keras_output_torch_tensor.shape)

<class 'numpy.ndarray'>
(1, 502, 502, 1)
<class 'numpy.ndarray'>
tensor([[[[False, False, False,  ..., False, False, False],
          [False, False,  True,  ..., False, False, False],
          [False,  True,  True,  ..., False, False, False],
          ...,
          [False, False, False,  ..., False, False, False],
          [False, False, False,  ..., False, False, False],
          [False, False, False,  ..., False, False,  True]],

         [[False, False,  True,  ..., False, False, False],
          [False, False,  True,  ..., False,  True, False],
          [False, False, False,  ..., False, False, False],
          ...,
          [False, False, False,  ..., False, False, False],
          [ True, False, False,  ...,  True, False, False],
          [False, False,  True,  ..., False, False, False]],

         [[False, False, False,  ..., False, False,  True],
          [False, False, False,  ..., False,  True, False],
          [False, False,  True,  ..., False, False, False],
 

In [17]:
#查看一下卷积层输出
print(torch_out[0,0,:3,:3])
print(keras_output_torch_tensor[0,0,:3,:3])

tensor([[ 0.0029,  0.0674, -0.0985],
        [ 0.0930,  0.0055, -0.0258],
        [ 0.0597, -0.2533,  0.3096]], grad_fn=<SliceBackward0>)
tensor([[ 0.0029,  0.0674, -0.0985],
        [ 0.0930,  0.0055, -0.0258],
        [ 0.0597, -0.2533,  0.3096]])


In [18]:
#查看卷积核
# print(keras_model.get_weights()[0])
print(keras_model.get_weights()[0][:,:,0,0].shape)
print([k for k in model.named_parameters()][0][1][0,0,:,:].data.shape)
print(keras_model.get_weights()[0][:,:,0,0])
print([k for k in model.named_parameters()][0][1][0,0,:,:].data)
#权重除了精度不一致没什么问题

(3, 3)
torch.Size([3, 3])
[[-0.02934367  0.05818463 -0.0313947 ]
 [ 0.02616983 -0.06751775 -0.01991458]
 [ 0.06146571 -0.04916605  0.00540901]]
tensor([[-0.0293,  0.0582, -0.0314],
        [ 0.0262, -0.0675, -0.0199],
        [ 0.0615, -0.0492,  0.0054]])


In [19]:
keras_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 502, 502, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 502, 502, 64) 640         input_1[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 502, 502, 64) 0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 502, 502, 64) 36928       lambda_1[0][0]                   
__________________________________________________________________________________________________
lambda_2 (

In [20]:
intermediate_layer_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 502, 502, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 502, 502, 64)      640       
Total params: 640
Trainable params: 640
Non-trainable params: 0
_________________________________________________________________
